In [ ]:
# !pip install llama_index
# !pip install langchain
# !pip install openai
# !pip install PyPDF2
# !pip install python-pptx
# !pip install docx2txt 
# !pip install sentencepiece

In [18]:
from llama_index import (
    GPTVectorStoreIndex,
    ResponseSynthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
import os
import openai
import json

from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader('./repo').load_data()
print(documents)

def open_file(filepath):
    with open(filepath, 'r') as infile:
        return infile.read()

class Chatbot:
    def __init__(self, api_key, index):
        self.index = index
        openai.api_key = api_key
        self.chat_history = []

    def generate_response(self, user_input):
        prompt = "\n".join([f"{message['role']}: {message['content']}" for message in self.chat_history[-5:]])
        prompt += f"\nUser: {user_input}"
        query_engine = index.as_query_engine()
        response = query_engine.query(user_input)
        # response = index.query(user_input)

        message = {"role": "assistant", "content": response.response}
        self.chat_history.append({"role": "user", "content": user_input})
        self.chat_history.append(message)
        return message
    
    def load_chat_history(self, filename):
        try:
            with open(filename, 'r') as f:
                self.chat_history = json.load(f)
        except FileNotFoundError:
            pass

    def save_chat_history(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.chat_history, f)

openai.api_key = open_file('./data/openaiapikey.txt')          
os.environ['OPENAI_API_KEY'] = openai.api_key

index = GPTVectorStoreIndex.from_documents(documents)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


[ImageDocument(text='<s_menu><s_nm> REASONS WHY</s_nm><s_unitprice> 66% 50%</s_unitprice><s_cnt> 31%</s_cnt><s_price> 66%</s_price><sep/><s_nm> BETER INFORMED CUSTOMERS MAKE THE MOST ORYOUR PRODUCTOR SERVICE</s_nm><s_unitprice> 56 87</s_unitprice><s_cnt> 32%</s_cnt><s_price> 66%</s_price></s_menu><s_sub_total><s_subtotal_price> 66%</s_subtotal_price></s_sub_total><s_total><s_total_price> 66%</s_total_price><s_emoneyprice> Orchestra</s_emoneyprice></s_total>', doc_id='1d368ed7-c119-49b7-8361-caddfb286141', embedding=None, doc_hash='fe77dcd0b61bc5911ab3c48d2da7cbfb90fde91f0e6ebc9474ad7c8d0a1e0695', extra_info=None, image=None)]


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 196 tokens


In [19]:
import textwrap  # Import the textwrap module

# Swap out your index below for whatever knowledge base you want
bot = Chatbot(openai.api_key, index=index)
bot.load_chat_history("chat_history.json")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye"]:
        print("Bot: Goodbye!")
        bot.save_chat_history("chat_history.json")
        break
    response = bot.generate_response(user_input)
    # print(f"Bot: {response['content']}")
    # Wrap the text and print the wrapped content
    wrapped_content = textwrap.wrap(response['content'], width=90)
    print("User: ", user_input)
    print("Bot: ",)
    for line in wrapped_content:
        print(line)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 6 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 273 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


User:  what is this infographic about?
Bot: 
 This infographic appears to be about the reasons why customers should be better informed
about a product or service, and the associated costs. It includes the unit price, count,
and total price of the product or service.


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 7 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 259 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


User:  tell me more about the customer training
Bot: 
 It is not possible to answer this question with the given context information. The
context information provided does not contain any information about customer training.


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 6 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 308 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


User:  why customer training is important?
Bot: 
 Customer training is important because it helps customers become better informed about a
product or service, which can lead to more effective and efficient use of the product or
service. This can result in increased customer satisfaction, loyalty, and ultimately more
sales. Additionally, customer training can help to reduce customer support costs by
providing customers with the knowledge and skills to troubleshoot and resolve issues on
their own.
Bot: Goodbye!
